In [1]:
import numpy as np
from scipy.special import gamma
import matplotlib.pyplot as plt

In [4]:
# Simulate data
def Bern_gen(nobs, k, theta, seed):
    np.random.seed(seed)
    obs_list = []
    for i in range(k):
        X_i = np.random.binomial(1, theta, nobs)
        obs_list.append(X_i)
    return np.matrix(obs_list).T

In [15]:
def prob_DH1(X, α=0.001, β=0.01):
    """Return marginal likelihood for bernoulli data using the conjugate prior distribution Bernoulli-Beta
       @X: N records of data with k columns
       @α, β: hyperparmeter for Beta distribution
    """
    md = np.sum(X,axis=0)
    N = X.shape[0]
    nominator = np.array(gamma(α+β)*gamma(α+md))*np.array(gamma(β+N-md))
    denominator = gamma(α)*gamma(β)*gamma(α+β+N)
    return np.prod(nominator/denominator)

In [62]:
def linkage_matrix(X, α = 0.001):
    """Find two nodes that have highest probability to merge"""
    linkage_list = []
    maximum = 0.6
    dim = X.copy().shape[0]
    merge_dim = X.shape[0]
    obs_list = [i for i in range(1,dim+1)]
    while (maximum > 0.5):
        for i in obs_list:
            for j in obs_list:
                if (j>i):
                    if (i<=dim and j<=dim):
                        s, w = i-1, j-1
                        nk = 2
                        prob_DTi, prob_DTj = prob_DH1(X[s]), prob_DH1(X[w])
                        di, dj = α, α
                    elif (i<=dim and j>dim):
                        s = i-1
                        w = np.array(linkage_list[j-dim-1][:2]) - 1
                        nk = linkage_list[j-dim-1][3] + 1
                        prob_DTi, prob_DTj = prob_DH1(X[s]), linkage_list[j-dim-1][4]
                        di, dj = α, linkage_list[j-dim-1][5]
                    elif (i>dim and j>dim):
                        s = np.array(linkage_list[i-dim-1][:2])-1
                        w = np.array(linkage_list[j-dim-1][:2])-1
                        nk = linkage_list[i-dim-1][3] + linkage_list[j-dim-1][3]                        
                        prob_DTi, prob_DTj = linkage_list[i-dim-1][4], linkage_list[j-dim-1][4]
                        di, dj = linkage_list[i-dim-1][5], linkage_list[j-dim-1][5]
                                            
                    Dk_tmp = np.vstack((X[s],X[w]))

                    dk = α*gamma(nk)+di*dj

                    pik = α*gamma(nk)/dk
                    prob_DT = prob_DH1(Dk_tmp)*pik + prob_DTi * prob_DTj * di * dj / dk

                    rk = pik*prob_DH1(Dk_tmp)/prob_DT

                    if (rk > maximum):
                        maximum = rk
                        merge_i = i
                        merge_j = j
                        merge_prob_DTi = prob_DT.copy()
                        merge_Dk = Dk_tmp.copy()
                        merge_dk = dk
        
        #print (maximum, rk)
        merge_dim+=1
        obs_list.append(merge_dim)
        if (merge_i) in obs_list: obs_list.remove(merge_i)    #remove merged observations' idx from list
        if (merge_j) in obs_list: obs_list.remove(merge_j)    
        
        X = np.vstack((X,merge_Dk))
        nk = merge_Dk.shape[0]
        
        linkage_list.append([merge_i, merge_j, np.log(maximum/(1-maximum)), nk, merge_prob_DTi, merge_dk])
    return (linkage_list)
            

In [63]:
X_test = Bern_gen(20, 10, 0.3, 125)

In [ ]:
linkage_matrix(X=X_test)